Notebook to load data extract data from 'nba_api' endpoints and transfer into JSON files for later operations

# Retrieve player details and dump into JSON file 

In [ ]:
"""Retrieve player details and dump into JSON"""
# Player details retrieved: name (key), team, team abbreviation, player id, team id
from nba_api.stats.endpoints.playerindex import PlayerIndex
import json

# Empty dictionary to store our cleaned player info
players_dict = {}
# Create the instance
index = PlayerIndex()
# Returns endpoint
players = index.player_index
# Create dictionary from endpoint
raw_dict = players.get_dict()
# Iterate over each player and create a dictionary with desired details
for pl in raw_dict['data']:
    players_dict[f'{pl[2]} {pl[1]}'] = {
        'team': f'{pl[7]} {pl[8]}',
        'abbreviation': pl[9],
        'player_id': pl[0],
        'team_id': pl[4],
        'first_name': pl[2],
        'last_name': pl[1]
    }

# Place the restructured data in JSON
file = '/Files/player_details.json'
with open(file, 'w') as f:
    json.dump(players_dict, f, indent=3)

# Retrieve NBA team details from static data and dump into JSON file

In [ ]:
"""Retrieve NBA team details from static data and input into JSON file"""
# Team details retrieved: name (key), team id, team abbreviation, nickname, city, state
from nba_api.stats.static import teams
import json

nba_teams = {}

all_teams = teams.get_teams()

# Iterate over team data and restructure into 'nba_teams' dict
for team in all_teams:
    nba_teams[team['full_name']] = {
        'id': team['id'],
        'abbreviation': team['abbreviation'],
        'nickname': team['nickname'],
        'city': team['city'],
        'state': team['state'],
    }

# Dump nba_teams into JSON
file = '/Files/team_details.json'
with open(file, 'w') as f:
    json.dump(nba_teams, f, indent=3)

# Load player traditional stats from endpoint
##### Changed the way player stats are loaded

In [ ]:
from nba_api.stats.endpoints.leaguedashplayerstats import LeagueDashPlayerStats
import json

# Load the players that we will need to iterate over (includes name, player_id, team_id) from JSON
with open('Files/player_details.json', 'r') as f:
    player_details = json.load(f)

# Get raw data from endpoint
raw_data = LeagueDashPlayerStats()
# Identify headers
player_headers = raw_data.get_dict()['resultSets'][0]['headers']
# Identify result sets
row_set = raw_data.get_dict()['resultSets'][0]['rowSet']
# modify the headers to lower case
lowered_headers = [hdr.lower() for hdr in player_headers]

# Empty dict with restructured player season data - Dumped into JSON at the end
player_stats = {}

for player in row_set:
    # Match the headers to the result sets, convert to player dictionary
    player_data = dict(zip(lowered_headers, player[:32]))
    # Add the current season
    player_data['season'] = '2024-25'
    # Remove unwanted fields
    player_data.pop('nickname')
    player_data.pop('team_abbreviation')
    player_data.pop('age')
    # Create sub dictionary for each player
    current_player = player_data['player_name']
    player_stats[current_player] = player_data

# Output player_stats dictionary into JSON
output_file = 'Files/player_data_new.json'
with open(output_file, 'w') as f:
    json.dump(player_stats, f, indent=3)

print('Complete')

# Load team traditional stats from endpoint

In [ ]:
# Team Dashboards, team statistics
# Same process above of retrieving player data/statistics
from nba_api.stats.endpoints.teamdashboardbygeneralsplits import TeamDashboardByGeneralSplits as tdbgs
from tqdm import tqdm
import json
import time

team_stats = {}

with open('/Files/team_details.json', 'r') as f:
    team_details = json.load(f)

for team, details in tqdm(team_details.items(), desc='Loading teams...'):
    team_splits = tdbgs(team_id=details['id'], timeout=60)
    current_team = team_splits.overall_team_dashboard.get_dict()

    team_stats[team] = {
        'team_id': details['id'],
        'abbreviation': details['abbreviation'],
        'team_name': team,
        'season': current_team['data'][0][2],
        'gp': current_team['data'][0][3],
        'wins': current_team['data'][0][4],
        'losses': current_team['data'][0][5],
        'win_pct': current_team['data'][0][6],
        'minutes': current_team['data'][0][7],
        'fgm': current_team['data'][0][8],
        'fga': current_team['data'][0][9],
        'fg_pct': current_team['data'][0][10],
        'fg3m': current_team['data'][0][11],
        'fg3a': current_team['data'][0][12],
        'fg3_pct': current_team['data'][0][13],
        'ftm': current_team['data'][0][14],
        'fta': current_team['data'][0][15],
        'ft_pct': current_team['data'][0][16],
        'oreb': current_team['data'][0][17],
        'dreb': current_team['data'][0][18],
        'reb': current_team['data'][0][19],
        'ast': current_team['data'][0][20],
        'tov': current_team['data'][0][21],
        'stl': current_team['data'][0][22],
        'blk': current_team['data'][0][23],
        'blka': current_team['data'][0][24],
        'pf': current_team['data'][0][25],
        'pfd': current_team['data'][0][26],
        'pts': current_team['data'][0][27],
        'plus_minus': current_team['data'][0][28]
    }

    time.sleep(1)

with open('/Files/team_data.json', 'w') as f:
    json.dump(team_stats, f, indent=3)

print('Complete')
